In [ ]:
# useful additional packages 
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx

from qiskit import Aer
from qiskit.tools.visualization import plot_histogram
from qiskit.optimization.applications.ising import max_cut, tsp
#from qiskit.aqua.operators import Z2Symmetries
#from qiskit.aqua.input import EnergyInput
from qiskit.aqua.algorithms import VQE, ExactEigensolver, NumPyEigensolver, NumPyMinimumEigensolver 
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.circuit.library import EfficientSU2, TwoLocal
from qiskit.optimization.algorithms import MinimumEigenOptimizer
from qiskit.optimization.converters import IsingToQuadraticProgram
from qiskit.optimization.problems import QuadraticProgram
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.device.basic_device_model import basic_device_noise_model

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
set_qiskit_aqua_logging(logging.INFO)  # choose INFO, DEBUG to see the log
from qiskit.providers.jobstatus import JobStatus

In [ ]:
from qiskit import IBMQ

provider = IBMQ.load_account()

In [ ]:
# Generating a graph of 3 nodes
n = 3
num_qubits = n ** 2

#ins = tsp.random_tsp(n, seed=123)
ins = tsp.parse_tsplib_format('/home/pui/tsp_sample/3cities.tsp')
print('distance\n', ins.w)

G = nx.Graph()
G.add_nodes_from(np.arange(0, ins.dim, 1))
colors = ['r' for node in G.nodes()]
for i in range(0, ins.dim):
    for j in range(i+1, ins.dim):
        G.add_edge(i, j, weight=ins.w[i,j])
pos = {k: v for k, v in enumerate(ins.coord)}

#default_axes = plt.axes(frameon=True)
#nx.draw_networkx(G, node_color=colors, node_size=600, alpha=.8, ax=default_axes, pos=pos)
#print(ins)

def draw_graph(G, colors, pos):
    default_axes = plt.axes(frameon=True)
    nx.draw_networkx(G, node_color=colors, node_size=600, alpha=.8, ax=default_axes, pos=pos)
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edge_labels)
    
draw_graph(G, colors, pos)


Brute force approach

In [ ]:
from itertools import permutations

def brute_force_tsp(w, N):
    a=list(permutations(range(1,N)))
    last_best_distance = 1e10
    for i in a:
        distance = 0
        pre_j = 0
        for j in i:
            distance = distance + w[j,pre_j]
            pre_j = j
        distance = distance + w[pre_j,0]
        order = (0,) + i
        if distance < last_best_distance:
            best_order = order
            last_best_distance = distance
            print('order = ' + str(order) + ' Distance = ' + str(distance))
    return last_best_distance, best_order
  
best_distance, best_order = brute_force_tsp(ins.w, ins.dim)
print('Best order from brute force = ' + str(best_order) + ' with total distance = ' + str(best_distance))

def draw_tsp_solution(G, order, colors, pos):
    G2 = nx.DiGraph()
    G2.add_nodes_from(G)
    n = len(order)
    for i in range(n):
        j = (i + 1) % n
        G2.add_edge(order[i], order[j], weight=G[order[i]][order[j]]['weight'])
    default_axes = plt.axes(frameon=True)
    nx.draw_networkx(G2, node_color=colors, edge_color='b', node_size=600, alpha=.8, ax=default_axes, pos=pos)
    edge_labels = nx.get_edge_attributes(G2, 'weight')
    nx.draw_networkx_edge_labels(G2, pos, font_color='b', edge_labels=edge_labels)
    
draw_tsp_solution(G, best_order, colors, pos)

Mapping to the Ising problem
-Generate Hamiltonian for TSP of a graph
 Input : ins (TspData) : TSP data including coordinates and distances.
 Return : tuple(WeightedPauliOperator, float): operator for the Hamiltonian and a constant shift for the obj function.

In [ ]:
#qubitOp, offset = tsp.get_operator(ins)
#algo_input = Z2Symmetries(qubitOp)
qubitOp, offset = tsp.get_operator(ins)
print('Offset:', offset)
print('Ising Hamiltonian:')
print(qubitOp.print_details())

In [ ]:
qp = QuadraticProgram()
qp.from_ising(qubitOp, offset, linear=True)
qp.to_docplex().prettyprint()

Checking that the full Hamiltonian gives the right cost

In [ ]:
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver())
result = exact.solve(qp)
print(result)

In [ ]:
#Making the Hamiltonian in its full form and getting the lowest eigenvalue and eigenvector
ee = NumPyMinimumEigensolver(qubitOp)
result = ee.run()

print('energy:', result.eigenvalue.real)
print('tsp objective:', result.eigenvalue.real + offset)
x = sample_most_likely(result.eigenstate)
print('feasible:', tsp.tsp_feasible(x))
z = tsp.get_tsp_solution(x)
print('solution:', z)
print('solution objective:', tsp.tsp_value(z, ins.w))
draw_tsp_solution(G, z, colors, pos)


In [ ]:
print(result)

Running it on quantum computer

We run the optimization routine using a feedback loop with a quantum computer that uses trial functions built with Y single-qubit rotations, 𝑈single(𝜃)=∏𝑛𝑖=1𝑌(𝜃𝑖)
, and entangler steps 𝑈entangler.

Run on statevector simulator

In [ ]:
seed = 10598
#seed = 1
backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)

spsa = SPSA(max_trials=200)
#ry = RY.(qubitOp.num_qubits, depth=5, entanglement='linear')
ry = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', reps=5, entanglement='linear')
#form = EfficientSU2(qubitOp.num_qubits, su2_gates=['ry'], entanglement='linear')
#vqe = VQE(qubitOp, ry, spsa)
#vqe = VQE(qubitOp, form, spsa)
vqe = VQE(qubitOp, ry, spsa, quantum_instance=quantum_instance)

result = vqe.run(quantum_instance)
#display(form.draw())
#print(form.qasm())

print('energy:', result.eigenvalue.real)
print('tsp objective:', result.eigenvalue.real + offset)
x = sample_most_likely(result.eigenstate)
print('feasible:', tsp.tsp_feasible(x))
z = tsp.get_tsp_solution(x)
print('solution:', z)
print('solution objective:', tsp.tsp_value(z, ins.w))
draw_tsp_solution(G, z, colors, pos)


In [ ]:
print(result)

In [ ]:
#create minimum eigen optimizer based on VQE and solve quadratic program

seed = 10598
backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)
# create minimum eigen optimizer based on VQE
vqe_optimizer = MinimumEigenOptimizer(vqe)

# solve quadratic program
result = vqe_optimizer.solve(qp)
print(result)

z = tsp.get_tsp_solution(x)
print('solution:', z)
print('solution objective:', tsp.tsp_value(z, ins.w))
draw_tsp_solution(G, z, colors, pos)

Run on qasm simulator

In [ ]:
# run quantum algorithm with shots

seed = 10598
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

spsa = SPSA(max_trials=500)
#ry = RY(qubitOp.num_qubits, depth=5, entanglement='linear')
ry = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', reps=5, entanglement='linear')
#form = EfficientSU2(qubitOp.num_qubits, su2_gates=['ry'], entanglement='linear')
#vqe = VQE(qubitOp, ry, spsa)
#vqe = VQE(qubitOp, form, spsa)
vqe = VQE(qubitOp, ry, spsa, quantum_instance=quantum_instance)

result = vqe.run(quantum_instance)

print('energy:', result.eigenvalue.real)
print('tsp objective:', result.eigenvalue.real + offset)
x = sample_most_likely(result.eigenstate)
print('feasible:', tsp.tsp_feasible(x))
z = tsp.get_tsp_solution(x)
print('solution:', z)
print('solution objective:', tsp.tsp_value(z, ins.w))
draw_tsp_solution(G, z, colors, pos)


In [ ]:
print(result)

In [ ]:
result['eigenstate']

In [ ]:
plot_histogram(result['eigenstate'])

In [ ]:
ry.qasm(filename='/home/pui/tsp.txt')

In [ ]:
#create minimum eigen optimizer based on VQE and solve quadratic program

seed = 10598
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)
# create minimum eigen optimizer based on VQE
vqe_optimizer = MinimumEigenOptimizer(vqe)

# solve quadratic program
result = vqe_optimizer.solve(qp)
print(result)

z = tsp.get_tsp_solution(x)
print('solution:', z)
print('solution objective:', tsp.tsp_value(z, ins.w))
draw_tsp_solution(G, z, colors, pos)

Run on real device

In [ ]:
# run quantum algorithm with shots

seed = 10598
backend = provider.get_backend('ibmq_qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

spsa = SPSA(max_trials=300)
ry = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', reps=5, entanglement='linear')
vqe = VQE(qubitOp, ry, spsa, quantum_instance=quantum_instance)


#ry = RY(qubitOp.num_qubits, depth=5, entanglement='linear')
#form = EfficientSU2(qubitOp.num_qubits, su2_gates=['ry'], entanglement='linear')
#vqe = VQE(qubitOp, ry, spsa)
#vqe = VQE(qubitOp, form, spsa)

result = vqe.run(quantum_instance)

print('energy:', result.eigenvalue.real)
print('tsp objective:', result.eigenvalue.real + offset)
x = sample_most_likely(result.eigenstate)
print('feasible:', tsp.tsp_feasible(x))
z = tsp.get_tsp_solution(x)
print('solution:', z)
print('solution objective:', tsp.tsp_value(z, ins.w))
draw_tsp_solution(G, z, colors, pos)

In [ ]:
result['eigenstate']

In [ ]:
plot_histogram(result['eigenstate'])

In [ ]:
# run quantum algorithm with shots

seed = 10598
backend = provider.get_backend('ibmq_16_melbourne')
#noise_model = basic_device_noise_model(provider.get_backend('ibmq_16_melbourne').properties())
noise_model = NoiseModel.from_backend(backend)
basis_gates = noise_model.basis_gates
coupling_map = backend.configuration().coupling_map
simulator = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(simulator, noise_model=noise_model, basis_gates=basis_gates, coupling_map=coupling_map, shots=1024, seed_simulator=seed, seed_transpiler=seed, skip_qobj_validation=False)

spsa = SPSA(max_trials=10000)
#ry = RY(qubitOp.num_qubits, depth=5, entanglement='linear')
ry = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', reps=5, entanglement='linear')
#vqe = VQE(qubitOp, ry, spsa)
vqe = VQE(qubitOp, ry, spsa, quantum_instance=quantum_instance)

#backend = provider.get_backend('ibmq_16_melbourne')

#noise_model = NoiseModel.from_backend(backend)
#basis_gates = noise_model.basis_gates
#coupling_map = backend.configuration().coupling_map


result = vqe.run(quantum_instance)

print('energy:', result['eigenvalue'])
print('time:', result['optimizer_time'])
#print('tsp objective:', result['energy'] + offset)
x = sample_most_likely(result['eigenstate'])
#x = tsp.sample_most_likely(result['eigvecs'][0])
print('feasible:', tsp.tsp_feasible(x))
z = tsp.get_tsp_solution(x)
print('solution:', z)
print('solution objective:', tsp.tsp_value(z, ins.w))

draw_tsp_solution(G, z, colors, pos)

In [ ]:
plot_histogram(result['eigenstate'])

[Optional] Checking that the full Hamiltonian made by docplex.get_operator gives the right cost

In [ ]:
from docplex.mp.model import Model
from qiskit.optimization.ising import docplex

# Create an instance of a model and variables
mdl = Model(name='tsp')
x = {(i,p): mdl.binary_var(name='x_{0}_{1}'.format(i,p)) for i in range(n) for p in range(n)}

# Object function
tsp_func = mdl.sum(ins.w[i,j] * x[(i,p)] * x[(j,(p+1)%n)] for i in range(n) for j in range(n) for p in range(n))
mdl.minimize(tsp_func)

# Constrains
for i in range(n):
    mdl.add_constraint(mdl.sum(x[(i,p)] for p in range(n)) == 1)
for p in range(n):
    mdl.add_constraint(mdl.sum(x[(i,p)] for i in range(n)) == 1)

In [ ]:
qubitOp_docplex, offset_docplex = docplex.get_operator(mdl)

In [ ]:
ee = NumPyEigensolver(qubitOp_docplex, k=1)
result = ee.run()

print('energy:', result['energy'])
print('tsp objective:', result['energy'] + offset_docplex)

x = sample_most_likely(result['eigvecs'][0])
print('feasible:', tsp.tsp_feasible(x))
z = tsp.get_tsp_solution(x)
print('solution:', z)
print('solution objective:', tsp.tsp_value(z, ins.w))
draw_tsp_solution(G, z, colors, pos)